# 9.- Azure ML Deploy Training

In [1]:
import yaml

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml import Input, Output
from azure.ai.ml import command

In [2]:
# Load configuration from the YAML file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [3]:
subscription_id = config["azure"]["subscription_id"]
resource_group_name = config["azure"]["resource_group_name"]
workspace_name = config["azure"]["workspace_name"]

environment_name = config["azure"]["environment_name"]
training_gpu_cluster = config["azure"]["training_gpu_cluster"]

data_asset_path = config["train"]["data_asset_path"]

In [4]:
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group_name,
    workspace_name
)

In [5]:
data_input = Input(
    path=config["train"]["data_asset_path"],
    type="uri_folder"
)

model_input = Input(
    path="../train/pretrained_models/yolo11n-seg.pt",
    type="uri_file"
)

In [6]:
output = Output(
    type="uri_folder",
    mode="upload"
)

In [13]:
command_job = """cp ${{inputs.data}}/data.yaml /tmp/data.yaml && \
sed -i "s|path:.*$|path: ${{inputs.data}}|" /tmp/data.yaml && \
python train.py \
--model ${{inputs.model}} \
--epochs 1 \
--batch 2 \
--output_dir ${{outputs.output_dir}}"""

In [14]:
job = command(
    code="../src",  # carpeta donde está train.py
    command=command_job,
    inputs={
        "data": data_input,
        "model": model_input
    },
    outputs={
        "output_dir": output
    },
    environment=f"{environment_name}@latest",
    compute=training_gpu_cluster,
    experiment_name="yolo11-segmentation-training",
    display_name="yolo11-segmentation-training"
)

In [ ]:
# Submit the job to Azure ML
job = ml_client.jobs.create_or_update(job)

In [22]:
config["train"]["job_name"] = job['name']

In [23]:
with open("../config.yaml", "w") as f:
    yaml.safe_dump(config, f, default_flow_style=False)